***Plot the probability distributions of the pose of a marker***
Each observation implies a marker position in the world frame, if the nominal robot model is assumed exact. This script calculates the would be pose of the marker from each observation, and plots a curve approximateing an underlying PDF

In [1]:
import pickle
import numpy as np
from parameter_estimator import ParameterEstimator
import utils
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from robot import *
from scipy.spatial.transform import Rotation as R
from pytransform3d.transform_manager import *

In [2]:
# import nominal positions of markers
with open('points.p', 'rb') as f:
    pointlist = pickle.load(f)  # deserialize using load()

nominal_locations = {}
nominal_locations_rounded = {}
for i, point in enumerate(np.array(pointlist).T):
    nominal_locations[i+1] = np.concatenate((point[0:3], np.zeros(1), point[3:5]))
    nominal_locations_rounded[i+1] = np.around(nominal_locations[i+1], 4)
    print(f"{i+1}: {nominal_locations[i+1]}")
#     print(f"{i+1}: {nominal_locations_rounded[i+1]}\n")

    

1: [1.93185165 0.         0.51763809 0.         0.26179939 0.        ]
2: [1.41421356 0.         1.41421356 0.         0.78539816 0.        ]
3: [0.51763809 0.         1.93185165 0.         1.30899694 0.        ]
4: [1.2044898  1.51038244 0.51763809 0.         0.26179939 0.8975979 ]
5: [0.88174773 1.10567669 1.41421356 0.         0.78539816 0.8975979 ]
6: [0.32274207 0.40470576 1.93185165 0.         1.30899694 0.8975979 ]
7: [-0.42987743  1.8834161   0.51763809  0.          0.26179939  1.7951958 ]
8: [-0.31469212  1.37875628  1.41421356  0.          0.78539816  1.7951958 ]
9: [-0.11518531  0.50465982  1.93185165  0.          1.30899694  1.7951958 ]
10: [-1.7405382   0.83819902  0.51763809  0.          0.26179939  2.6927937 ]
11: [-1.27416239  0.61360427  1.41421356  0.          0.78539816  2.6927937 ]
12: [-0.4663758   0.22459475  1.93185165  0.          1.30899694  2.6927937 ]
13: [-1.7405382  -0.83819902  0.51763809  0.          0.26179939  3.5903916 ]
14: [-1.27416239 -0.61360427  1

In [19]:
from pytransform3d.transformations import concat

for marker_id in nominal_locations.keys():
    print(marker_id)
    # transform to marker
    location = nominal_locations[marker_id]
    print(location)
    r = R.from_euler('xyz', location[3:])
    rot = r.as_matrix()
    t = location[0:3]
    H = utils.merge_H_transform(rot, t)
    #utils.roundprint(H)
    
    # generate some relative poses for the camera to go, a
    for i in range(1):
        #cam_rel_t = np.concatenate([np.random.uniform(-0.1,0.1,(1,)), np.array([-0.3]), np.random.uniform(-0.1,0.1,(1,))])
        cam_rel_t = np.concatenate([np.array([-0.3, 0, 0])])
        #cam_rel_r = np.random.uniform(-0.07,0.07,(3,))
        cam_rel_r = np.zeros(3)
        r = R.from_euler('xyz', cam_rel_r)
        rot = r.as_matrix()
        t = cam_rel_t
        cam_rel_H = utils.merge_H_transform(rot, t)

        utils.roundprint(cam_rel_H)
        
        # apply these relative transforms to the poses
        cam_pose = concat(H, cam_rel_H)
        utils.roundprint(cam_pose)
        
        # split into translation and euler 
        rot, tvec = utils.split_H_transform(cam_pose)
        r = R.from_matrix(rot)
        rotvec = r.as_euler('zxy')
        location = np.concatenate([tvec, rotvec])
        print(location)

1
[1.93185165 0.         0.51763809 0.         0.26179939 0.        ]
 1.000  0.000  0.000  0.300 
 0.000  1.000  0.000  0.000 
 0.000  0.000  1.000  0.000 
 0.000  0.000  0.000  1.000 


 0.966  0.000  0.259  2.232 
 0.000  1.000  0.000  0.000 
-0.259  0.000  0.966  0.518 
 0.000  0.000  0.000  1.000 


[2.23185165 0.         0.51763809 0.         0.         0.26179939]
2
[1.41421356 0.         1.41421356 0.         0.78539816 0.        ]
 1.000  0.000  0.000  0.300 
 0.000  1.000  0.000  0.000 
 0.000  0.000  1.000  0.000 
 0.000  0.000  0.000  1.000 


 0.707  0.000  0.707  1.714 
 0.000  1.000  0.000  0.000 
-0.707  0.000  0.707  1.414 
 0.000  0.000  0.000  1.000 


[1.71421356 0.         1.41421356 0.         0.         0.78539816]
3
[0.51763809 0.         1.93185165 0.         1.30899694 0.        ]
 1.000  0.000  0.000  0.300 
 0.000  1.000  0.000  0.000 
 0.000  0.000  1.000  0.000 
 0.000  0.000  0.000  1.000 


 0.259  0.000  0.966  0.818 
 0.000  1.000  0.000  0.000 
-0.966